In [1]:
import geopandas as gpd
import pandas as pd
from tqdm import tqdm

### Merge MA Census Tract Shapefiles with census data from social explorer

In [29]:
#only read these in again if you don't have the ma census tracts files


census_data = pd.read_csv('../data/census_data_by_tract.csv') #relevant statistics from social explorer, by census tract
us_tracts_gdf = gpd.read_file("../data/TRACT_2019_US_SL140__2019-11-18_12-56-55-677/TRACT_2019_US_SL140_Coast_Clipped.shp", crs="EPSG:26986") # all US census tracts
ma_tracts_gdf = us_tracts_gdf[us_tracts_gdf['STATEFP'] == '25'].sort_values(by="GEOID") # filter out just MA census tracts



ma_tracts_data_gdf = ma_tracts_gdf.merge(census_data, left_on='GEOID', right_on='FIPS', how='left') #left merge because not all of the census tract geos have census data from social explorer
ma_tracts_data_gdf.to_file("../data/ma_tracts_census_data.geojson", driver='GeoJSON')

In [ ]:
new_data = gpd.read_file('../data/preprocessed_data/station_buffer_census_bgs_separate.geojson')


In [ ]:
new_data.iloc[0:500].explore()


In [30]:
census_tract_df = gpd.read_file('../data/ma_tracts_census_data.geojson')
census_tract_df = census_tract_df.to_crs(crs="EPSG:26986")
stops_with_buffer = gpd.read_file('../data/mbta_stops.geojson')
stops_with_buffer = stops_with_buffer.to_crs(crs="EPSG:26986")


In [40]:
# Compute area of census bg
census_tract_df['tract_area_km2'] = census_tract_df.area / 1e6

In [19]:
mbtac_gdf = gpd.read_file("../data/mbta_municipalities.geojson")
mbtac_gdf = mbtac_gdf.to_crs(crs="EPSG:26986")

In [31]:
assert census_tract_df.crs == stops_with_buffer.crs

In [33]:
columns_to_weight = [
    'Households:', #denomenator for income categories
    'Households: Less than $25,000',
    'Households: $25,000 to $49,999',
    'Households: $50,000 to $74,999',
    'Households: $75,000 to $99,999',
    'Households: $100,000 or More',
    'Total Population', # this is the total population of the census tract (count)
    'Total Population: Male: Under 18 Years',
    'Total Population: Male: 18 to 34 Years',
    'Total Population: Male: 35 to 64 Years',
    'Total Population: Male: 65 Years and Over',
    'Total Population: Female',
    'Total Population: Female: Under 18 Years',
    'Total Population: Female: 18 to 34 Years',
    'Total Population: Female: 35 to 64 Years',
    'Total Population: Female: 65 Years and Over',
    'Total Population: Male', # denomenator for age categories beginning with male
    'Total Population: Female', # denomenator for age categories beginning with female
    'Total Population: Not Hispanic or Latino',
    'Total Population: Not Hispanic or Latino: White Alone',
    'Total Population: Not Hispanic or Latino: Black or African American Alone',
    'Total Population: Not Hispanic or Latino: American Indian and Alaska Native Alone',
    'Total Population: Not Hispanic or Latino: Asian Alone',
    'Total Population: Not Hispanic or Latino: Native Hawaiian and Other Pacific Islander Alone',
    'Total Population: Not Hispanic or Latino: Some Other Race Alone',
    'Total Population: Not Hispanic or Latino: Two or More Races',
    'Total Population: Hispanic or Latino', # merging all racial categories within "Hispanic and Latino" so that 1) there aren't an overwhelming number of racial categories 
    # 2) those categories are not very descriptive anyway (the way the U.S. census handles Hispanic/Latino people is weird)
    'Median Household Income (In 2022 Inflation Adjusted Dollars)',
    'Workers 16 Years and Over:', # total number of workers 16 years and older and should be used as the denomenator for the different commute method categories after weighting
    'Workers 16 Years and Over: Car, Truck, or Van',
    'Workers 16 Years and Over: Drove Alone',
    'Workers 16 Years and Over: Public Transportation (Includes Taxicab)',
    'Workers 16 Years and Over: Motorcycle',
    'Workers 16 Years and Over: Bicycle',
    'Workers 16 Years and Over: Walked',
    'Workers 16 Years and Over: Other Means',
    'Workers 16 Years and Over: Worked At Home',
    'Occupied Housing Units', # total number of housing units/households per tract. should be used as the denomenator for the different # of vehicles per hh categories after weighting
    'Occupied Housing Units: No Vehicle Available',
    'Occupied Housing Units: 1 Vehicle Available',
    'Occupied Housing Units: 2 Vehicles Available', # removed the last couple of categories with large numbers of vehicles bc not relevant.
]

columns_to_weight_ints = [i for i in columns_to_weight if '%' not in i]
columns_to_weight_perc = [i for i in columns_to_weight if '%' in i]
renamed_columns = ['weighted_' + str(i) for i in columns_to_weight]


In [37]:
census_tract_df[columns_to_weight].astype(float).fillna(0)


,Households:,"Households: Less than $25,000","Households: $25,000 to $49,999","Households: $50,000 to $74,999","Households: $75,000 to $99,999","Households: $100,000 or More",Total Population,Total Population: Male: Under 18 Years,Total Population: Male: 18 to 34 Years,Total Population: Male: 35 to 64 Years,...,Workers 16 Years and Over: Public Transportation (Includes Taxicab),Workers 16 Years and Over: Motorcycle,Workers 16 Years and Over: Bicycle,Workers 16 Years and Over: Walked,Workers 16 Years and Over: Other Means,Workers 16 Years and Over: Worked At Home,Occupied Housing Units,Occupied Housing Units: No Vehicle Available,Occupied Housing Units: 1 Vehicle Available,Occupied Housing Units: 2 Vehicles Available
0,1996.0,327.0,290.0,241.0,214.0,924.0,3630.0,255.0,333.0,1255.0,...,39.0,0.0,193.0,315.0,49.0,730.0,1996.0,245.0,1129.0,476.0
1,1975.0,137.0,324.0,264.0,281.0,969.0,4352.0,411.0,230.0,873.0,...,0.0,0.0,0.0,82.0,0.0,415.0,1975.0,151.0,653.0,852.0
2,1031.0,174.0,263.0,16.0,74.0,504.0,1627.0,32.0,35.0,432.0,...,0.0,0.0,0.0,2.0,44.0,300.0,1031.0,48.0,349.0,477.0
3,1363.0,80.0,326.0,293.0,107.0,557.0,2739.0,56.0,121.0,536.0,...,15.0,0.0,0.0,0.0,0.0,268.0,1363.0,12.0,529.0,564.0
4,1361.0,144.0,188.0,297.0,217.0,515.0,2985.0,209.0,277.0,448.0,...,15.0,0.0,11.0,0.0,22.0,157.0,1361.0,29.0,576.0,495.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1468,1445.0,285.0,203.0,256.0,141.0,560.0,3443.0,282.0,285.0,668.0,...,2.0,9.0,0.0,0.0,1.0,117.0,1445.0,44.0,445.0,543.0
1469,2010.0,390.0,316.0,398.0,223.0,683.0,4985.0,595.0,644.0,1154.0,...,0.0,0.0,0.0,0.0,0.0,184.0,2010.0,145.0,913.0,541.0
1470,2234.0,97.0,391.0,281.0,241.0,1224.0,6023.0,536.0,691.0,1309.0,...,139.0,0.0,0.0,0.0,68.0,504.0,2234.0,176.0,841.0,858.0
1471,1299.0,28.0,233.0,250.0,138.0,650.0,3615.0,468.0,539.0,608.0,...,30.0,5.0,0.0,99.0,12.0,350.0,1299.0,47.0,424.0,542.0


In [41]:
# Assume 'stops_with_buffer' and 'census_bg_df' are defined and are valid GeoDataFrames

# Initialize an empty GeoDataFrame with a specified CRS
station_buffer_census_df = gpd.GeoDataFrame()

# Loop through each station in 'stops_with_buffer'
for idx, station in tqdm(stops_with_buffer.iterrows()):
    # Create a GeoDataFrame for the current station's buffer geometry
    curr_gdf = gpd.GeoDataFrame({'geometry': [station['geometry']]}, crs=stops_with_buffer.crs)

    # Clip the census blocks to the current station's buffer
    curr_census = census_tract_df.clip(curr_gdf)

    # Project the clipped census blocks to the desired CRS
    curr_census = curr_census.to_crs(crs="EPSG:26986")

    # Calculate the area in square kilometers
    curr_census['clipped_area_km2'] = curr_census['geometry'].area / 1e6
    
    # Calculate the respective area percent
    curr_census['station_buffer_area_km2'] = station['geometry'].area / 1e6
    curr_census['percent_of_buffer_area'] = curr_census['clipped_area_km2'] / curr_census['station_buffer_area_km2']

    # Calculate the percent of the block group area
    curr_census['percent_of_census_tract_area'] = curr_census['clipped_area_km2'] / curr_census['tract_area_km2']
    
    # Recompute columns to reflect the spatial weighting with respect to the census block group
    # Columns that are in absolute values can just be multiplied by the area that is in the station buffer
    weighted_census = curr_census[columns_to_weight].copy()
    weighted_census.loc[:, columns_to_weight_ints] = weighted_census[columns_to_weight_ints].mul(curr_census['percent_of_census_tract_area'], axis=0)
    weighted_census.loc[:, columns_to_weight_ints] = weighted_census[columns_to_weight_ints].round(decimals=0).astype(int)
    # The columns that are in percent, however, have to be weighted by the percent of buffer area that is covered by that tract
    weighted_census.loc[:, columns_to_weight_perc] = curr_census[columns_to_weight_perc].mul(curr_census['percent_of_buffer_area'], axis=0)

    # Make sure that we do not have any percent that are bigger than 100% !!!
    assert (weighted_census[[i for i in weighted_census.columns if '%' in i]] > 100).any().any() == False
    
    # Finally, we assign the calculated values to the original census
    curr_census[renamed_columns] = weighted_census.rename(columns={o: n for o, n in zip(columns_to_weight, renamed_columns)})
    
    # We can now just add all the values in the curr_census and add the row to the respective station entry
    summed_census = curr_census[renamed_columns].sum(axis=0)
    stops_with_buffer.loc[stops_with_buffer.index==idx, renamed_columns] = summed_census[renamed_columns].to_numpy()

    # Add an identifier for the current station (e.g., the loop index or the station's index from 'stops_with_buffer')
    curr_census['stop_id'] = station['stop_id']

    # Append the current census blocks to the main GeoDataFrame
    station_buffer_census_df = pd.concat([station_buffer_census_df, curr_census], ignore_index=True)


0it [00:00, ?it/s]


TypeError: can't multiply sequence by non-int of type 'float'

In [ ]:
brookline_stations = station_buffer_census_df.sjoin(mbtac_gdf[mbtac_gdf['municipality'] == 'Brookline'])['stop_id'].unique()
station_buffer_census_df[station_buffer_census_df['stop_id'].isin(brookline_stations)].head(5)

In [ ]:
station_buffer_census_df_complete = pd.merge(station_buffer_census_df, stops_with_buffer[[i for i in stops_with_buffer.columns if i not in renamed_columns]], how='left', on='stop_id')
station_buffer_census_df_complete['geometry'] = station_buffer_census_df_complete['geometry_x']
del station_buffer_census_df_complete['geometry_x']
del station_buffer_census_df_complete['geometry_y']
station_buffer_census_df_complete = gpd.GeoDataFrame(station_buffer_census_df_complete)

In [ ]:
station_buffer_census_df_complete[station_buffer_census_df_complete['stop_id']=='WR-0329-02'].explore()

##TODO: translate absolute value columns to pct and rename like weighted_pct_lowercase _ separated column name

In [ ]:
station_buffer_census_df_complete.to_file('../data/preprocessed_data/station_buffer_census_tracts_separate.geojson')
stops_with_buffer.to_file('../data/preprocessed_data/station_buffer_census_cumulative.geojson')

You can find the above files on GDrive:

- `station_buffer_census_bgs_separate.geojson`: [GDrive Link](https://drive.google.com/file/d/1AAlo7GSMCMSBJOBQTCzS9KA7ZcD2m9uI/view?usp=drive_link)
- `station_buffer_census_cumulative.geojson`: [GDrive Link](https://drive.google.com/file/d/181sMgDXKSQLBNk647Dw_5VApV3WfjGP1/view?usp=drive_link)